In [ ]:
import gym
import gym_foa
from gym import wrappers

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
from collections import namedtuple
import random
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import copy
import time
import os
import shutil

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

N = 1
EXPERIENCE_CAPACITY = 7*int(1e3)
MAX_EPI = 1*int(1e3)
LR_actor = .001
LR_critic = .001
BATCH_SIZE = 64

exp_name = 'pendulum_version_alr_{}_clr_{}_bs_{}_capa_{}k_epi_{}k_x{}'.format(\
            LR_actor, LR_critic, BATCH_SIZE, float(EXPERIENCE_CAPACITY)/1000., float(MAX_EPI)/1000., N)
exp_name = 'test'
if os.path.exists(exp_name):
    shutil.rmtree(exp_name)
os.mkdir(exp_name)

ENV_NAME = 'foa-v0'
# test
ENV_NAME = 'Pendulum-v0'
env = gym.make(ENV_NAME)
RECORD = True

if RECORD:
    #env = wrappers.Monitor(env, '{}/record'.format(exp_name), force=True, video_callable=lambda episode_id: True)
    env = wrappers.Monitor(env, '{}/record'.format(exp_name), force=True)
TAU = .01
GAMMA = .99


TARGET_UPDATE_FREQUENCY = 200

MAX_STEP = 100

EPSILON = 1
EPSILON_DECAY = 1./(1e5)
N_STATES = env.observation_space.shape[0]
N_ACTIONS = env.action_space.shape[0]
N_UAVS = 1
V_MAX = env.action_space.high[0]
print V_MAX

In [ ]:
# initialization
use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
Tensor = FloatTensor

Transition = namedtuple('Transition', ('state', 'action', 'reward', 'next_state'))

In [ ]:
uf_lb = -3.*(1e-3)
uf_ub = 3.*(1e-3)

def fanin_init(size, fanin=None):
    fanin = fanin or size[0]
    v = 1. / np.sqrt(fanin)
    return torch.Tensor(size).uniform_(-v, v)


# classes
class Experience(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.mem = []
        self.pos = 0
    
    def push(self, o, a, r, o_next):
        if len(self.mem) < self.capacity:
            self.mem.append(None)
        self.mem[self.pos] = Transition(o, a, r, o_next)
        self.pos = (self.pos + 1) % self.capacity
    
    def sample(self, size):
        return random.sample(self.mem, min(size, len(self.mem)))
        #return self.mem[:1]
    
class Critic(nn.Module):
    def __init__(self):
        super(Critic, self).__init__()
        self.fcS = nn.Linear(N_STATES, 400)
        #nn.init.kaiming_normal(self.fcS.weight)
        #nn.init.uniform(self.fcS.weight, uf_lb, uf_ub)
        self.fcS.weight.data = fanin_init(self.fcS.weight.data.size())
        self.bnS = nn.BatchNorm1d(400)
        
        self.fcA = nn.Linear(N_ACTIONS, 400)
        #nn.init.kaiming_normal(self.fcA.weight)
        #nn.init.uniform(self.fcA.weight, uf_lb, uf_ub)
        self.fcA.weight.data = fanin_init(self.fcA.weight.data.size())
        self.bnA = nn.BatchNorm1d(400)
        
        self.fc1 = nn.Linear(400, 300)
        #nn.init.kaiming_normal(self.fc1.weight)
        #nn.init.uniform(self.fc1.weight, uf_lb, uf_ub)
        self.fc1.weight.data = fanin_init(self.fc1.weight.data.size())
        self.bn1 = nn.BatchNorm1d(300)
        
        self.fc2 = nn.Linear(300, 1)
        #nn.init.kaiming_normal(self.fc2.weight)
        nn.init.uniform(self.fc2.weight, uf_lb, uf_ub)
    
    def forward(self, x, y):
#         s = F.relu(self.bnS(self.fcS(x)))
#         a = F.relu(self.bnA(self.fcA(y)))
#         o = F.relu(self.bn1(self.fc1(s+a)))
#         o = self.fc2(o)
               
        s = (self.fcS(x))
        a = (self.fcA(y))
        o = F.relu(s+a)
        o = (self.fc1(o))
        o = F.relu(o)
        o = (self.fc2(o))
        
        return o

class Actor(nn.Module):
    def __init__(self):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(N_STATES/N_UAVS, 400)
        #nn.init.kaiming_normal(self.fc1.weight)
        #nn.init.uniform(self.fc1.weight, uf_lb, uf_ub)
        self.fc1.weight.data = fanin_init(self.fc1.weight.data.size())
        self.bn1 = nn.BatchNorm1d(400)
        
        self.fc2 = nn.Linear(400, 300)
        #nn.init.kaiming_normal(self.fc2.weight)
        #nn.init.uniform(self.fc2.weight, uf_lb, uf_ub)
        self.fc2.weight.data = fanin_init(self.fc2.weight.data.size())
        self.bn2 = nn.BatchNorm1d(300)
        
        self.fc3 = nn.Linear(300, N_ACTIONS/N_UAVS)
        #nn.init.kaiming_normal(self.fc3.weight)
        nn.init.uniform(self.fc3.weight, uf_lb, uf_ub)
        
    def forward(self, x):
#         x = F.relu(self.bn1(self.fc1(x)))
#         x = F.relu(self.bn2(self.fc2(x)))
#         x = self.fc3(x)
        
        x = (self.fc1(x))
        x = F.relu(x)
        x = (self.fc2(x))
        x = F.relu(x)
        x = (self.fc3(x))
        x = F.tanh(x)
        return x

In [ ]:
# [reference] https://github.com/matthiasplappert/keras-rl/blob/master/rl/random.py

class RandomProcess(object):
    def reset_states(self):
        pass

class AnnealedGaussianProcess(RandomProcess):
    def __init__(self, mu, sigma, sigma_min, n_steps_annealing):
        self.mu = mu
        self.sigma = sigma
        self.n_steps = 0

        if sigma_min is not None:
            self.m = -float(sigma - sigma_min) / float(n_steps_annealing)
            self.c = sigma
            self.sigma_min = sigma_min
        else:
            self.m = 0.
            self.c = sigma
            self.sigma_min = sigma

    @property
    def current_sigma(self):
        sigma = max(self.sigma_min, self.m * float(self.n_steps) + self.c)
        return sigma


# Based on http://math.stackexchange.com/questions/1287634/implementing-ornstein-uhlenbeck-in-matlab
class OrnsteinUhlenbeckProcess(AnnealedGaussianProcess):
    def __init__(self, theta, mu=0., sigma=1., dt=1e-2, x0=None, size=1, sigma_min=None, n_steps_annealing=1000):
        super(OrnsteinUhlenbeckProcess, self).__init__(mu=mu, sigma=sigma, sigma_min=sigma_min, n_steps_annealing=n_steps_annealing)
        self.theta = theta
        self.mu = mu
        self.dt = dt
        self.x0 = x0
        self.size = size
        self.reset_states()

    def sample(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + self.current_sigma * np.sqrt(self.dt) * np.random.normal(size=self.size)
        self.x_prev = x
        self.n_steps += 1
        return x

    def reset_states(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros(self.size)

In [ ]:
def map_to_v(a, low, high):
    return (low+high)/2 + a*(high-low)/2

def choose_action(state, actor, rand_proc=None):
    global epsilon
    a = actor(Variable(Tensor(state).unsqueeze(0))).data.cpu().numpy()[0].astype('float64')
    if rand_proc is not None:
        a += max(epsilon, 0)*rand_proc.sample()
        a = np.clip(a, -1., 1.)
        epsilon -= EPSILON_DECAY
    #print epsilon
    return a

def update_actor_critic(target_actor, target_critic, \
                       actor, critic, exp, optim_actor, optim_critic):
    
    
    
    # sample minibatch
    minibatch = Transition(*zip(*exp.sample(BATCH_SIZE)))
    bat_o = Variable(Tensor(minibatch.state))
    bat_a = Variable(Tensor(minibatch.action))
    bat_r = Variable(Tensor(minibatch.reward)).unsqueeze(1)
    bat_o_ = Variable(Tensor(minibatch.next_state))
    
    bat_a0_ = target_actor(bat_o_[:,:N_STATES/N_UAVS])
    #bat_a1_ = target_actor(bat_o_[:,N_STATES/N_UAVS:-N_STATES/N_UAVS])
    #bat_a2_ = target_actor(bat_o_[:,-N_STATES/N_UAVS:])
    #bat_a_o_ = torch.cat([bat_a0_, bat_a1_, bat_a2_], dim=1)
    bat_a_o_ = bat_a0_

    Gt = bat_r + GAMMA * target_critic(bat_o_, bat_a_o_)
    Gt.detach_()
    eval_o = critic(bat_o, bat_a)
    criterion = nn.MSELoss()
    if use_cuda:
        criterion.cuda()
    loss = criterion(eval_o, Gt)
    #optimizer = optim.Adam(critic.parameters(), lr=LR_critic)
    optim_critic.zero_grad()
    loss.backward()
    optim_critic.step()
    
    # update actor
    bat_a0 = actor(bat_o[:,:N_STATES/N_UAVS])
    #bat_a1 = actor(bat_o[:,N_STATES/N_UAVS:-N_STATES/N_UAVS])
    #bat_a2 = actor(bat_o[:,-N_STATES/N_UAVS:])
    #bat_a_o = torch.cat([bat_a0, bat_a1, bat_a2], dim=1)
    bat_a_o = bat_a0
    
    obj = torch.mean(critic(bat_o, bat_a_o))
    #optimizer = optim.Adam(actor.parameters(), lr=-LR_actor)
    optim_actor.zero_grad()
    obj.backward()
    optim_actor.step()    

def update_target(target_actor, target_critic, \
                         actor, critic):
    target_actor.load_state_dict(actor.state_dict())
    target_critic.load_state_dict(critic.state_dict())

def soft_update(target, behavior, tau):
    for key in target.state_dict().keys():
        target.state_dict()[key].copy_(tau * behavior.state_dict()[key] + (1-tau) * target.state_dict()[key])

In [ ]:


n_vec_r = []
n_vec_avg_cost = []
n_vec_total_cost = []
n_vec_target_update = []
n_vec_experience_refresh = []
n_vec_training_time = []
for n in xrange(N):    
    start_time = time.time()
    
    target_actor = Actor()
    target_critic = Critic()
    target_actor.eval()
    target_critic.eval()
    
    actor = Actor()
    critic = Critic()
    if use_cuda:
        target_actor.cuda()
        target_critic.cuda()
        actor.cuda()
        critic.cuda()
    exp = Experience(EXPERIENCE_CAPACITY)
    optim_critic = optim.Adam(critic.parameters(), lr=LR_critic)
    optim_actor = optim.Adam(actor.parameters(), lr=-LR_actor)

    target_actor.load_state_dict(actor.state_dict())
    target_critic.load_state_dict(critic.state_dict())
    
    random_process = OrnsteinUhlenbeckProcess(\
            size=N_ACTIONS/N_UAVS, theta=.15, mu=0, sigma=.2)
    epsilon = EPSILON
    
    vec_r = []
    vec_avg_cost = []
    vec_total_cost = []
    vec_target_update = []
    vec_experience_refresh = []
    update_counter = 0
    for epi in xrange(MAX_EPI):
        if epi%(MAX_EPI/10)==0:
            print 'n:{}, epi:{}'.format(n, epi)
                    
        random_process.reset_states()
        
        o = env.reset()
        acc_r = 0
        
        local_r = []
        
        avg_cost = np.zeros([4])
        total_cost = 0
    
        counter = 0
        #for t in xrange(MAX_STEP):    
        while True:
            counter += 1
            
            if RECORD:
                env.render()
            
            #actor.eval()
            
            a0 = choose_action(o[:N_STATES/N_UAVS], actor)
#             a0 = choose_action(o[:N_STATES/N_UAVS], actor, random_process)
            #a1 = choose_action(o[N_STATES/N_UAVS:-N_STATES/N_UAVS], actor)
            #a2 = choose_action(o[-N_STATES/N_UAVS:], actor)
            #a = np.hstack([a0, a1, a2])
            a = a0
            
#             # ground truth
#             if update_counter<1000:
# #                 a = np.array([0.,3./5.])
#                 a = np.array([0.,3.])
            
            o_, r, done, info = env.step(map_to_v(a, -V_MAX, V_MAX))
#            o_, r, done, info = env.step(a)
    
            exp.push(o, a, r, o_)
            
#             cost = np.array(info['cost'])
#             avg_cost += (cost-avg_cost)/(counter)
            total_cost += r
            
            actor.train()
            
            update_actor_critic(target_actor, target_critic, \
                               actor, critic, exp, optim_actor, optim_critic)
            update_counter += 1
            
            if update_counter % TARGET_UPDATE_FREQUENCY == 0:
                vec_target_update.append(epi)
                update_target(target_actor, target_critic, \
                             actor, critic)
            if update_counter % EXPERIENCE_CAPACITY == 0:
                vec_experience_refresh.append(epi)
            
            local_r.append(r)
            acc_r += r
            o = o_
            if done:
                break
        
        vec_r.append(acc_r)
        vec_avg_cost.append(avg_cost)
        vec_total_cost.append(total_cost)
        
    n_vec_r.append(vec_r)
    n_vec_avg_cost.append(vec_avg_cost)
    n_vec_total_cost.append(vec_total_cost)
    n_vec_target_update.append(vec_target_update)
    n_vec_experience_refresh.append(vec_experience_refresh)
    n_vec_training_time.append((time.time() - start_time)/60)
    
    torch.save(actor.state_dict(), '{}/actor{}.pt'.format(exp_name, n))
    
    print "--- {} minutes ---".format((time.time() - start_time)/60)



In [ ]:
def plot_avg_1d(vec, title=None, xlabel=None, ylabel=None, save=False, folder='.',\
                ylim=None, begin=0, marker='', target=None, exp=None):
    record = np.array(vec)
    if len(record.shape) is not 1:
        print 'vec should be 1D array [e0, e1, ...]'
        return
    if ylim is not None:
        axes = plt.gca()
        axes.set_ylim(ylim)
    mu = record.mean(axis=0)
    plt.plot(np.array(range(record.shape[0]))+begin, record, marker)
    plt.plot(np.array(range(record.shape[0]))+begin, mu*np.ones(record.shape[0]), color='red')
    patches = []
    if target is not None and len(target)>0:
        target = np.array(target)
        target_color = 'b'
        target_patch = mpatches.Patch(color=target_color, label='update target')
        patches.append(target_patch)
        for t in target:
            plt.axvline(t, ls='dashed', c=target_color)
    if exp is not None and len(exp)>0:
        exp = np.array(exp)
        exp_color = 'r'
        exp_patch = mpatches.Patch(color=exp_color, label='exp refresh')
        patches.append(exp_patch)
        for e in exp:
            plt.axvline(e, ls='dashed', c=exp_color)
    if len(patches)>0:
        plt.legend(handles=patches)
    if title is not None:
        plt.title(title)
    if xlabel is not None:
        plt.xlabel(xlabel)
    if ylabel is not None:
        plt.ylabel(ylabel, rotation=45)
    if save:
        plt.savefig('{}/{}.png'.format(folder, title))      
    plt.show()

def plot_avg_2d(vec, title=None, xlabel=None, ylabel=None, save=False, folder='.',\
                ylim=None, begin=0):
    record = np.array(vec)
    if len(record.shape) is not 2:
        print 'vec should be 2D array [[seq0], [seq1], ...]'
        return
    if ylim is not None:
        axes = plt.gca()
        axes.set_ylim(ylim)
    mu = record.mean(axis=0)
    sigma = record.std(axis=0)
    lower_bound = mu-sigma
    upper_bound = mu+sigma
    plt.plot(np.array(range(mu.shape[0]))+begin, mu, color='red')
    plt.fill_between(np.array(range(mu.shape[0]))+begin, lower_bound, upper_bound, facecolor='blue', alpha=0.5)
    if title is not None:
        plt.title(title)
    if xlabel is not None:
        plt.xlabel(xlabel)
    if ylabel is not None:
        plt.ylabel(ylabel, rotation=45)
    if save:
        plt.savefig('{}/{}.png'.format(folder, title))
    plt.show()

# data
n_vec_avg_cost = np.array(n_vec_avg_cost)
n_vec_total_cost = np.array(n_vec_total_cost)
n_vec_target_update = np.array(n_vec_target_update)
n_vec_experience_refresh = np.array(n_vec_experience_refresh)


#cost = gym_foa.envs.FoaEnv.Cost(*np.rollaxis(n_vec_avg_cost,2))
#avg_cost = cost.collision[0]+cost.goal[0]+cost.formation[0]+cost.v_pref[0]

# param
save = True
marker = '.'

target = np.array([10,20,30])
experience = np.array([15,25])

plot_avg_2d(n_vec_total_cost, title='total cost', ylim=[-8000,1000], save=save, folder=exp_name)
for i in xrange(n_vec_avg_cost.shape[0]):
    plot_avg_1d(n_vec_total_cost[i], title='total cost of {} \n training time: {} minutes'.format(i, n_vec_training_time[i]),\
                ylim=[-8000,1000], save=save, folder=exp_name)
                #target=n_vec_target_update[i], )
                #exp=n_vec_experience_refresh[i])

# plot_avg_2d(n_vec_total_cost, title='total cost', save=save, folder=exp_name)
# for i in xrange(n_vec_avg_cost.shape[0]):
#     plot_avg_1d(n_vec_total_cost[i], title='total cost of {} \n training time: {} minutes'.format(i, n_vec_training_time[i]),\
#                 save=save, folder=exp_name)
#                 #target=n_vec_target_update[i], )
#                 #exp=n_vec_experience_refresh[i])

# marker = ''
# int0 = slice(0,500)
# plot_avg_1d(n_vec_total_cost[0][int0], title='total cost: first drop', ylim=[-8000,1000], begin=int0.start, save=save, marker=marker)
#plot_avg_1d(cost.v_pref[0][int0], title='v_pref: first drop', begin=int0.start, save=save, marker=marker)


In [ ]:
#torch.save(actor.state_dict(), 'actor_3w.param')
#torch.save(critic.state_dict(), 'critic.param')

#from gym import wrappers

#env = wrappers.Monitor(env, 'exp', force=True)


actor = Actor()
actor.cuda()
#actor.eval()
actor.train()
actor.load_state_dict(torch.load('actor0.pt'))
print actor.state_dict()

s = np.ones(N_STATES/N_UAVS)
s = Variable(Tensor(s)).unsqueeze(0)
#print s
# for _ in xrange(10):
#     print actor(s)
#     print actor.state_dict()['bn1.running_mean'][:10]



#actor.eval()
#actor.cuda()

# start_time = time.time()
# counter = 0
# for n in xrange(10):
#     o = env.reset()
#     for t in xrange(MAX_STEP):
#         counter += 1
# #        print 'n:{}, t:{}'.format(n, t)
# #         env.render()
#         #a0 = choose_action(o[:N_STATES/N_UAVS], actor)
        
        
#         a0 = actor(Variable(torch.FloatTensor(o[:N_STATES/N_UAVS]).unsqueeze(0))).data.cpu().numpy()[0].astype('float64')
        
        
#         #a1 = choose_action(o[N_STATES/N_UAVS:-N_STATES/N_UAVS], actor)
#         #a2 = choose_action(o[-N_STATES/N_UAVS:], actor)
#         #a = np.hstack([a0, a1, a2])
        
# #        print a0
        
#         o_, r, done, info = env.step(a0)
    
#         o = o_
#         if done:
#             break
# (time.time() - start_time)/counter

In [ ]:
import pickle

name = 'capa_7.5k_epi_2w_x5'

# Saving the objects:
with open('{}_total_cost.pickle'.format(name), 'w') as f:  # Python 3: open(..., 'wb')
    pickle.dump(n_vec_total_cost, f)

with open('{}_target_update.pickle'.format(name), 'w') as f:  # Python 3: open(..., 'wb')
    pickle.dump(n_vec_target_update, f)

with open('{}_experience_refresh.pickle'.format(name), 'w') as f:  # Python 3: open(..., 'wb')
    pickle.dump(n_vec_experience_refresh, f)
    
    
## Getting back the objects:
#with open('objs.pickle') as f:  # Python 3: open(..., 'rb')
#    k = pickle.load(f)
    
#print cost